## Python для работы с базами данных

Изменения в Vagrantfile:
1. Перед запуском виртуалки раскомментировать строку 50
```bash
config.vm.network "forwarded_port", guest: 5432, host: 5432, host_ip: "127.0.0.1"
```

2. На виртуалке меняем в конфигурационном файле postgresql метод аутентификации (в редакторе nano после внесения изменений нажимаем Ctrl+x и на вопрос отвечаем y + enter):
```bash
sudo nano /etc/postgresql/10/main/pg_hba.conf
```

Находим строчку (в конце файла)
```bash
# Database administrative login by Unix domain socket
local   all             postgres                                peer
```

меняем peer на md5
```bash
local   all             postgres                                md5
```

и разрешаем подключения извне (меняем 127.0.0.1/32 на 0.0.0.0/0)
```bash
# IPv4 local connections:
host    all             all             0.0.0.0/0            md5
```

В файле конфигураций разрешаем слушать все IP-адреса
```bash
sudo nano /etc/postgresql/10/main/postgresql.conf
# находим строчку с listen_addresses и меняем значение на
listen_addresses = '*'
```

В конце перезапускаем postgresql
```bash
sudo service postgresql restart
```

3. Заходим под пользователем postgres и добавляем в базу пользователя с правами на чтение
```bash
sudo su -l postgres
psql -U postgres -c "CREATE ROLE readaccess;"
psql -U postgres -c "GRANT SELECT ON ALL TABLES IN SCHEMA public TO readaccess;"
psql -U postgres -c "ALTER DEFAULT PRIVILEGES IN SCHEMA public GRANT SELECT ON TABLES TO readaccess;"
psql -U postgres -c "CREATE USER ds WITH PASSWORD 'ds';"
psql -U postgres -c "GRANT readaccess TO ds;"
```

In [1]:
import psycopg2

In [2]:
# инициализируем соединение с базой данных
conn = psycopg2.connect("dbname='anyclass_prod' user='analytic' host='10.10.3.87' password='I59vPF59sbc6o9Lly'")

In [3]:
cur = conn.cursor()

In [ ]:
# для больших запросов
cur.itersize = 10000

In [4]:
# отправляем запрос
cur.execute('SELECT COUNT(*) FROM project_1."order";')

In [5]:
# получаем все строки ответа
cur.fetchall()

[(156064,)]

In [6]:
cur.execute('SELECT * FROM ratings order by userid limit 10;')

In [7]:
for row in cur:
    print(row)

(1, 147, 4.5, 1425942435)
(1, 858, 5.0, 1425941523)
(1, 1221, 5.0, 1425941546)
(1, 1246, 5.0, 1425941556)
(1, 1968, 4.0, 1425942148)
(1, 2762, 4.5, 1425941300)
(1, 2918, 5.0, 1425941593)
(1, 2959, 4.0, 1425941601)
(1, 4226, 4.0, 1425942228)
(1, 110, 1.0, 1425941529)


### Немного про запросы в MySQL

In [ ]:
# для тяжелых запросов к MySQL можно использовать SSDictCursor
import pymysql

connection = pymysql.connect(
    host = 'dbhost',
    port = 3306,
    user = 'dbuser',
    password = 'dbpass',
    db = 'db',
    cursorclass = pymysql.cursors.SSDictCursor
)

with connection.cursor() as cursor:
    cursor.execute(sqlQuery)

    for line in cursor:
        print(line)

### Базы данных и pandas

In [8]:
import pandas as pd

In [9]:
data = pd.read_sql('SELECT * FROM ratings limit 10;', conn)
data

,userid,movieid,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
5,1,1968,4.0,1425942148
6,1,2762,4.5,1425941300
7,1,2918,5.0,1425941593
8,1,2959,4.0,1425941601
9,1,4226,4.0,1425942228


## Персональные данные

In [10]:
# плохой вариант для аналитики
database_rows = [
    {'name': 'Наталья', 'email': 'nata123@mail.ru', 'ip': '127.0.0.1'},
    {'name': 'Михаил', 'email': 'mikha@yandex.ru', 'ip': '127.0.0.1'},
]

In [11]:
import hashlib

In [14]:
hashlib.algorithms_available

{'DSA',
 'DSA-SHA',
 'MD4',
 'MD5',
 'MDC2',
 'RIPEMD160',
 'SHA',
 'SHA1',
 'SHA224',
 'SHA256',
 'SHA384',
 'SHA512',
 'blake2b',
 'blake2s',
 'dsaEncryption',
 'dsaWithSHA',
 'ecdsa-with-SHA1',
 'md4',
 'md5',
 'mdc2',
 'ripemd160',
 'sha',
 'sha1',
 'sha224',
 'sha256',
 'sha384',
 'sha3_224',
 'sha3_256',
 'sha3_384',
 'sha3_512',
 'sha512',
 'shake_128',
 'shake_256',
 'whirlpool'}

In [12]:
email = 'username@yandex.ru'

In [13]:
m = hashlib.sha256()

m.update(bytes(email, 'utf-8'))
m.hexdigest()

'2388d885ffef200977eefc0a585a578bc30e00cc0b1cde0678ed49ef04659d51'

In [30]:
email = 'username@yandex.ru.'
m = hashlib.sha256()

m.update(bytes(email, 'utf-8'))
m.hexdigest()

'45c7a3ed1571a6251b4909ae982af6a1066ad94681788e63e7ac2679b097c5a7'

### Передача сообщений
Проблема [Man in the middle](https://ru.wikipedia.org/wiki/%D0%90%D1%82%D0%B0%D0%BA%D0%B0_%D0%BF%D0%BE%D1%81%D1%80%D0%B5%D0%B4%D0%BD%D0%B8%D0%BA%D0%B0)

In [ ]:
message = 'Согласуете отпуск на следующей неделе?'

In [15]:
correct_answer = 'Да'
corrupted_answer = 'Нет'

Заранее договоримся с руководителем добавлять в конец каждого сообщения. Кроме вас и руководителя никто не должен знать это значение.

In [16]:
seed = '123'

К каждому сообщению добавляем seed и считаем хэш:

In [17]:
sending_message = correct_answer + seed
sending_message

'Да123'

Вместе с message передаем хэш

In [18]:
m = hashlib.sha256()

m.update(bytes(sending_message, 'utf-8'))
m.hexdigest()

'cc26051b81b3776c74ba22f0f8a903a0c2dec1ce3385cc40b774e0a0643ab177'

Если злоумышленник перехватит и изменит сообщение, то без seed он не сможет посчитать нужное значение хэша. Когда вы получаете сообщение, то добавляете к нему seed, считаете хэш и сравниваете с тем, что приложен к сообщению:

In [19]:
received_message = 'Нет'

In [20]:
m = hashlib.sha256()

m.update(bytes(received_message, 'utf-8'))
m.hexdigest()

'43e5ab70f691a6a6391be1da21bc18c31c04892ceb19fde88e44ad50c5ec9052'

Хэш получился другой, значит сообщение изменено!

В случае неизмененного сообщения хэши совпадут:

In [21]:
m = hashlib.sha256()

m.update(bytes(correct_answer + seed, 'utf-8'))
m.hexdigest()

'cc26051b81b3776c74ba22f0f8a903a0c2dec1ce3385cc40b774e0a0643ab177'

### Упражнение
Зашифруйте персональные данные в списке database_rows

In [35]:
database_rows = [
    {'name': 'Наталья', 'email': 'nata123@mail.ru', 'ip': '127.0.0.1'},
    {'name': 'Михаил', 'email': 'mikha@yandex.ru', 'ip': '127.0.0.1'},
    {'name': 'Наталья', 'email': 'nata123@mail.ru', 'ip': '127.0.0.1'},
]

In [36]:
db_sha = {}

for rec in database_rows:
    m = hashlib.sha256()

    m.update(bytes(rec['name'] + 'whlfeq', 'utf-8'))
    db_sha['name'] = m.hexdigest()
    
    print(db_sha)

{'name': 'bfed49f40befead4597ffbbae875e6a7c8cb35b682532a27189a5d18224faa7f'}
{'name': 'b7f9dd381133be6bd44bf8efae68ca29ee4404633f7f192bca5b69149b14a587'}
{'name': 'bfed49f40befead4597ffbbae875e6a7c8cb35b682532a27189a5d18224faa7f'}
